In [ ]:
pip install -q langchain_google_genai langchain langchain_community

In [ ]:
import os

# Set environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your_langchain_api_key_here"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "chatbot_with_langchain"  
os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest",convert_system_message_to_human=True)

In [ ]:
model.invoke("hi").content

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:
parser.invoke(model.invoke("hi"))

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
while True:
  message=input("write your query:")
  if message=="bye":
    print("good bye have a great day!")
    break
  else:
    print(parser.invoke(model.invoke([HumanMessage(content=message)])))






In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [ ]:
result=model.invoke(
    [
        HumanMessage(content="hi i am Sayan"),
        AIMessage(content="hello Sayan!what can i do for you?"),
        HumanMessage(content="what is my name?")
    ]
)

In [ ]:
parser.invoke(result)

In [ ]:
store={}

In [ ]:

def get_session_history(session_id:str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]=InMemoryChatMessageHistory()
  return store[session_id]

In [ ]:
config ={"configurable": {"session_id": "firstchat"}}

In [ ]:
model_with_memory=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
model_with_memory.invoke([HumanMessage(content="Hi! I'm Sayan")],config=config).content

In [ ]:
model_with_memory.invoke([HumanMessage(content="What is my name?")],config=config).content

In [ ]:
config={"configurable":{"session_id":"secondchat"}}

In [ ]:
model_with_memory.invoke([HumanMessage(content="What is my name?")],config=config).content

In [ ]:
config={"configurable":{"session_id":"firstchat"}}

In [ ]:
config

In [ ]:
store

In [ ]:
config={"configurable":{"session_id":"firstchat"}}

In [ ]:
model_with_memory.invoke([HumanMessage(content="What is my name?")],config=config).content

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

In [ ]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant.Answer all questions to the best of your ability.",),
        MessagesPlaceholder(variable_name="messages"),

    ]
)

In [ ]:
chain = prompt | model

In [ ]:
chain.invoke({"messages":["hi! I am Sayan"]})

In [ ]:
chain.invoke({"messages":[HumanMessage(content="What is my name?")]}).content

In [ ]:
config={"configurable":{"session_id":"thirdchat"}}

In [ ]:
response=model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Sayan"),
    ],config=config
)

In [ ]:
response.content

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
response=model_with_memory.invoke(
    [HumanMessage(content="What is  4+8?"),
    ],config=config
)
print(response.content)

In [ ]:
response=model_with_memory.invoke(
    [HumanMessage(content="What should i add in previous addition to make it 10?"),
    ],config=config
)
print(response.content)

In [ ]:
store

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

In [ ]:
trimmer = trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)


In [ ]:
messages=[

    HumanMessage(content="Hi! I'm Sayan"),
    AIMessage(content="Hi"),
    HumanMessage(content="I like chocalate ice-cream"),
    AIMessage("content=nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="You're welcome"),
    HumanMessage(content="bye"),
    AIMessage(content="bye"),
 ]

In [ ]:
model.get_num_tokens_from_messages(messages)

In [ ]:
trimmer.invoke(messages)

In [ ]:
trimmed_messages=trimmer.invoke(messages)
display(trimmed_messages)

In [ ]:
model.get_num_tokens_from_messages(trimmed_messages)

In [ ]:
prompt

In [ ]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant.Answer all questions to the best of your ability.",),
        MessagesPlaceholder(variable_name="messages"),

    ]
)

In [ ]:
trimmed_messages

In [ ]:
trimmed_messages + [HumanMessage(content="What is my Name?")]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain =(
    # RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) # Removed this line
    itemgetter("messages")|trimmer # Pass the trimmed messages directly
    |prompt
    |model # Moved model after prompt
)

response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What is my Name?")],
    "language": "English",
    }
)
response.content

In [ ]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)


In [ ]:
config = {"configurable": {"session_id": "fifthchat"}}

In [ ]:
response = model_with_memory.invoke(
  {"messages": messages + [HumanMessage(content="What is my name?")]},
  config=config
)
response.content